In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k
# from implicit.evaluation import AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекомендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [5]:
%%time

# your_code

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.01 µs


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [6]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [7]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision_at_k = flags.sum() / len(recommended_list)
    
    return precision_at_k

In [8]:
result_precision = result[['user_id']].copy()
result_precision['random_recommendation_precision'] = ''
result_precision['popular_recommendation_precision'] = ''
result_precision['itemitem_precision'] = ''
result_precision['cosine_precision'] = ''
result_precision['tfidf_precision'] = ''
result_precision['own_purchases_precision'] = ''

result_precision.head(2)

,user_id,random_recommendation_precision,popular_recommendation_precision,itemitem_precision,cosine_precision,tfidf_precision,own_purchases_precision
0,1,,,,,,
1,3,,,,,,


In [9]:
result.loc[result['user_id'] == 1, 'random_recommendation'].iloc[0].split(', ')[:5]

['[5586238', '1015228', '866118', '2416733', '2603573]']

In [10]:
result.loc[result['user_id'] == 1, 'actual'].iloc[0].split('[')[1].split()[:5]

['821867', '834484', '856942', '865456', '889248']

In [11]:
# Precision@5 for random_recommendation
for user_id in result['user_id']:
    recommended_list = result.loc[result['user_id'] == user_id, 'random_recommendation'].iloc[0].split(', ')
    bought_list = result.loc[result['user_id'] == user_id, 'actual'].iloc[0].split('[')[1].split()
    precision_at_5 = precision_at_k(recommended_list, bought_list, k=5)    
    result_precision.loc[result_precision['user_id'] == user_id, 'random_recommendation_precision'] = precision_at_5

In [12]:
random_recommendation_precision_avg = result_precision['random_recommendation_precision'].mean()
random_recommendation_precision_avg

0.00039177277179236047

In [13]:
precision_avg = {'random_recommendation_precision_avg': random_recommendation_precision_avg}
precision_avg

{'random_recommendation_precision_avg': 0.00039177277179236047}

In [14]:
# Precision@5 for popular_recommendation
for user_id in result['user_id']:
    recommended_list = result.loc[result['user_id'] == user_id, 'popular_recommendation'].iloc[0].split(', ')
    bought_list = result.loc[result['user_id'] == user_id, 'actual'].iloc[0].split('[')[1].split()
    precision_at_5 = precision_at_k(recommended_list, bought_list, k=5)    
    result_precision.loc[result_precision['user_id'] == user_id, 'popular_recommendation_precision'] = precision_at_5

In [15]:
popular_recommendation_precision_avg = result_precision['popular_recommendation_precision'].mean()
popular_recommendation_precision_avg

0.04701273261508361

In [16]:
precision_avg['popular_recommendation_precision_avg'] = popular_recommendation_precision_avg

In [17]:
# Precision@5 for itemitem
for user_id in result['user_id']:
    recommended_list = result.loc[result['user_id'] == user_id, 'itemitem'].iloc[0].split(', ')
    bought_list = result.loc[result['user_id'] == user_id, 'actual'].iloc[0].split('[')[1].split()
    precision_at_5 = precision_at_k(recommended_list, bought_list, k=5)    
    result_precision.loc[result_precision['user_id'] == user_id, 'itemitem_precision'] = precision_at_5

In [18]:
itemitem_precision_avg = result_precision['itemitem_precision'].mean()
itemitem_precision_avg

0.018021547502448564

In [19]:
precision_avg['itemitem_precision_avg'] = itemitem_precision_avg

In [20]:
# Precision@5 for cosine
for user_id in result['user_id']:
    recommended_list = result.loc[result['user_id'] == user_id, 'cosine'].iloc[0].split(', ')
    bought_list = result.loc[result['user_id'] == user_id, 'actual'].iloc[0].split('[')[1].split()
    precision_at_5 = precision_at_k(recommended_list, bought_list, k=5)    
    result_precision.loc[result_precision['user_id'] == user_id, 'cosine_precision'] = precision_at_5

In [21]:
cosine_precision_avg = result_precision['cosine_precision'].mean()
cosine_precision_avg

0.01968658178256612

In [22]:
precision_avg['cosine_precision_avg'] = cosine_precision_avg

In [23]:
# Precision@5 for tfidf
for user_id in result['user_id']:
    recommended_list = result.loc[result['user_id'] == user_id, 'tfidf'].iloc[0].split(', ')
    bought_list = result.loc[result['user_id'] == user_id, 'actual'].iloc[0].split('[')[1].split()
    precision_at_5 = precision_at_k(recommended_list, bought_list, k=5)    
    result_precision.loc[result_precision['user_id'] == user_id, 'tfidf_precision'] = precision_at_5

In [24]:
tfidf_precision_avg = result_precision['tfidf_precision'].mean()
tfidf_precision_avg

0.020959843290891306

In [25]:
precision_avg['tfidf_precision_avg'] = tfidf_precision_avg

In [26]:
# Precision@5 for own_purchases
for user_id in result['user_id']:
    recommended_list = result.loc[result['user_id'] == user_id, 'own_purchases'].iloc[0].split(', ')
    bought_list = result.loc[result['user_id'] == user_id, 'actual'].iloc[0].split('[')[1].split()
    precision_at_5 = precision_at_k(recommended_list, bought_list, k=5)    
    result_precision.loc[result_precision['user_id'] == user_id, 'own_purchases_precision'] = precision_at_5

In [27]:
own_purchases_precision_avg = result_precision['own_purchases_precision'].mean()
own_purchases_precision_avg

0.14511916421808504

In [28]:
precision_avg['own_purchases_precision_avg'] = own_purchases_precision_avg

In [31]:
best_quality = f'Best quality: {max(precision_avg, key=precision_avg.get)}' 
best_quality

'Best quality: own_purchases_precision_avg'

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [30]:
# your_code